# Busca precios de proveedor DYD en mercado libre y ordena por ganancia estimada

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from lxml import etree
!pip install tika
from tika import parser
from google.colab import files

# Obtener los nombres y precios de DYD

# Lectura
raw = parser.from_file('preferencial.pdf')
df = (raw['content'])
df = df.replace('\n',',')
df = df.replace(',,',',')
df = df.replace(',,,,,,,,,,,,,,,,,,,,','')

# Reemplazo de las comas en los precios (para luego dividir los productos por coma)
for i,v in enumerate(df):
  if i < len(df)-3:
    if v == ',':
      if df[i+2] == '0':
        df = df[:i] + '.' + df[i + 1:]

      try:
        valor1 = int(df[i+1])
        if type(valor1) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
        continue

      try:
        valor2 = int(df[i+2])
        if type(valor2) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
          continue

# Split
df = df.split(',')

# Obtener los codigos, precios y articulos
codigos_DYD = []
articulos_DYD = []
precios_DYD = []
for producto in df:
  if '$' in producto:
    articulos_DYD.append(producto)
    producto = producto.split(' ')
    precios_DYD.append(int(producto[-1]))
    codigo = 0
    for cod in producto:
      try:
        codigo = int(cod)  
        break
      except:
        continue
    if codigo > 0:
      codigos_DYD.append(codigo)
    else:
      codigos_DYD.append('Codigo no encontrado')

# Corregir nombres de articulos y obtencion de la version buscable
new_articulos_DYD = []
articulos_DYD_buscables = []
for producto in articulos_DYD:
  producto = producto.split('$')[0]
  producto = producto.split(' ')
  for i,v in enumerate(producto):
    try:
      codigo = int(v)
      del producto[i]
      break
    except:
      continue
  final = ' '.join(producto)
  final_buscable = '-'.join(producto)

  # Correcciones abreviasiones
  final_buscable = final_buscable.replace('NUTRI…', 'NUTRITION')
  final_buscable = final_buscable.replace('NUT…', 'NUTRITION')
  final_buscable = final_buscable.replace('EVOLUTI…', 'EVOLUTION')
  final_buscable = final_buscable.replace('%', '%25')

  # Apend
  new_articulos_DYD.append(final)
  articulos_DYD_buscables.append(final_buscable[:-1])
articulos_DYD = new_articulos_DYD.copy()

# Obtencion del precio en Mercado libre
lista_precios = []
url = 'https://listado.mercadolibre.com.ar/'
noBuscables = []
contador = 0
for i,prod in enumerate(articulos_DYD_buscables):
  r = requests.get(url+prod) 
  if r.status_code == 200:
    contador += 1
    soup = BeautifulSoup(r.content, 'html.parser')
    # Precios
    dom = etree.HTML(str(soup))
    precios = dom.xpath("//li[@class='ui-search-layout__item']//div[@class='ui-search-price__second-line']//span[@class='price-tag-fraction']")
    precios = [int(i.text.replace('.','')) for i in precios][:3]
    lista_precios.append(int(np.mean(precios)))
  else:
    print('\n'*10)
    print(f'NO PUDO ACCEDERSE A PAG PRODUCTO {articulos_DYD[i]}')
    print('\n'*10)
    noBuscables.append(articulos_DYD[i])
    del articulos_DYD[i]
    del precios_DYD[i]
    del codigos_DYD[i]
  print(f"Buscando articulo nro {contador} de {len(articulos_DYD_buscables)}")  

# Sustituto de descripciones (productos sin codigo en descripcion)
lista = [0]*(len(articulos_DYD) - len(lista_precios))
lista_precios.extend(lista)

# Data Frame
df = pd.DataFrame(np.array([articulos_DYD,precios_DYD,lista_precios, codigos_DYD]).T, columns=['Producto','precioDeCompra','precioEstimadoVenta','CodigoListaDYD'])
df.precioDeCompra = df['precioDeCompra'].astype(int)
df.precioEstimadoVenta = df['precioEstimadoVenta'].astype(int)
df['Ganancia'] = df['precioEstimadoVenta'] - df['precioDeCompra']
df = df.sort_values('Ganancia', ascending=False)

# Reubicar columnas
Producto = df.pop('Producto')
precioDeCompra = df.pop('precioDeCompra')
precioEstimadoVenta = df.pop('precioEstimadoVenta')
CodigoListaDYD = df.pop('CodigoListaDYD')
Ganancia = df.pop('Ganancia')

df.insert(0, 'Producto', Producto)
df.insert(1, 'Ganancia', Ganancia)
df.insert(2, 'precioDeCompra', precioDeCompra)
df.insert(3, 'precioEstimadoVenta', precioEstimadoVenta)
df.insert(4, 'CodigoListaDYD', CodigoListaDYD)

df.head(60)

In [ ]:
# <----- Apretar Play para descargarlo como excel
df.to_csv('Ganancias_DYD.csv')
files.download("Ganancias_DYD.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>